<a href="https://colab.research.google.com/github/vkthakur90/AI_Applications/blob/master/Positional_Trading_with_Auto_Regression_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
ticker = 'NIFTYBEES.NS'
prev_day = 0
scale_factor = 1.0e2

history_length = 800

ar_p = 100
ar_h = 50

num_paths = 1000
paths_to_plot = 1

risk_budget = 100000

In [4]:
def download_data(entity):
    print(f"Downloading {ticker}")

    # Download intraday data with a 1-minute interval.
    data = yf.download(ticker, period="5y", interval="1d")

    if data.index.tzinfo is None:
        data.index = data.index.tz_localize('UTC')

    data.index = data.index.tz_convert('Asia/Kolkata')

    data_ = pd.DataFrame({
        'S' : data['Close'].values.reshape((-1,))
    }, index = data.index)

    entity['data'] = data_

In [5]:
def calculate_return(entity):
    data = entity['data']

    data.eval('log_S = log(S)', inplace = True)
    data['r'] = scale_factor * data['log_S'].diff()

In [6]:
def feature_engg(entity):
    data = entity['data']

    features_df = data[['S', 'r']]

    for idx in range(ar_p):
        features_df[f'r_m{idx + 1}'] = features_df['r'].shift(idx + 1)

    for idx in range(ar_h):
        features_df[f'r_p{idx}'] = features_df['r'].shift(-idx)


    features_df.dropna(inplace = True)

    entity['features'] = features_df

In [7]:
def fit_ar_model(entity):
    formula_list = [f'r_p{idx} ~ ' + ' + '.join([f'r_m{idx + 1}' for idx in range(ar_p)]) for idx in range(ar_h)]

    entity['ar_models'] = [smf.ols(formula, data = entity['features']).fit() for formula in formula_list]

    entity['ar_params'] = pd.DataFrame({m.model.endog_names : m.params for m in entity['ar_models']})

    entity['ar_weights'] =  entity['ar_params'].iloc[1:]
    entity['ar_bias'] =  entity['ar_params'].iloc[:1]

    entity['ar_residue'] = pd.DataFrame({m.model.endog_names : m.resid for m in entity['ar_models']})

In [8]:
entity = {}
download_data(entity)
calculate_return(entity)
feature_engg(entity)
fit_ar_model(entity)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [9]:
pd.DataFrame({'sigma' : ((entity['ar_residue']**2).sum()/(len(entity['ar_residue']) - ar_p - 1)).apply(np.sqrt)})


,sigma
r_p0,0.822672
r_p1,0.824143
r_p2,0.823869
r_p3,0.824236
r_p4,0.825125
r_p5,0.825214
r_p6,0.824409
r_p7,0.824140
r_p8,0.823992
r_p9,0.824213
